<a href="https://colab.research.google.com/github/RyuichiSaito1/multilingual-economic-narratives/blob/main/notebooks/x_api_free_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

X API
https://docs.x.com/x-api/introduction

In [3]:
#!/usr/bin/env python3
"""
Free tier: 100 reads/month, 500 writes/month, basic v2 endpoints
"""

import requests
import json
from datetime import datetime

# Install requests if needed
try:
    import requests
except ImportError:
    !pip install requests

In [4]:
class SimpleXAPITester:

    def __init__(self, bearer_token: str):
        self.bearer_token = bearer_token
        self.headers = {"Authorization": f"Bearer {bearer_token}"}
        self.base_url = "https://api.twitter.com/2/tweets/search/recent"
        self.request_count = 0

    def search_tweets(self, query: str, lang: str = None, max_results: int = 10):

        # Build search query with language filter if specified
        search_query = query
        if lang:
            search_query += f" lang:{lang}"

        params = {
            "query": search_query,
            "max_results": min(max_results, 100),
            "tweet.fields": "created_at,lang,public_metrics"
        }

        # Add language filter as separate parameter
        if lang:
            params["lang"] = lang

        print(f"Query: '{query}'")
        if lang:
            print(f"Language: {lang}")
        print(f"Searching...")

        try:
            response = requests.get(self.base_url, headers=self.headers, params=params)
            self.request_count += 1

            print(f"Status: {response.status_code}")

            if response.status_code == 200:
                data = response.json()
                self.display_results(data, query, lang)
                return data
            else:
                print(f"Error: {response.text}")
                return None

        except Exception as e:
            print(f"Request failed: {e}")
            return None

    def display_results(self, data, query, lang):

        if "data" not in data or not data["data"]:
            print("No tweets found")
            return

        tweets = data["data"]
        print(f"\nFound {len(tweets)} tweets")

        # Language breakdown
        languages = {}
        for tweet in tweets:
            tweet_lang = tweet.get("lang", "unknown")
            languages[tweet_lang] = languages.get(tweet_lang, 0) + 1

        print(f"Languages: {languages}")

        # Display ALL tweets fetched (not just a sample)
        print(f"\nDisplaying ALL {len(tweets)} tweets found:")

        for i, tweet in enumerate(tweets, 1):
            text = tweet.get("text", "")
            tweet_lang = tweet.get("lang", "N/A")
            likes = tweet.get("public_metrics", {}).get("like_count", 0)

            print(f"\n{i}. [{tweet_lang}] {text[:100]}...")
            print(f"   Likes: {likes}")

        print(f"\nAPI requests used: {self.request_count}/100 (monthly limit)")

In [ ]:
def main():

    BEARER_TOKEN = "your_bearer_token_here"

    # Initialize tester
    tester = SimpleXAPITester(BEARER_TOKEN)

    # Economic keywords search in Finnish using OR logic
    economic_keywords_query = "price OR cost OR inflation OR deflation OR expensive OR cheap OR purchase OR sale"

    tester.search_tweets(
        query=economic_keywords_query,
        lang="fi",
        max_results=30
    )

    print(f"\nTotal API requests: {tester.request_count}")

if __name__ == "__main__":
    main()